# One Product

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

#Encontrando o html dos produtos
soup = BeautifulSoup(page.text, 'html.parser')
products = soup.find('ul', class_ = 'products-listing small')
products_list = products.find_all('article', class_= 'hm-product-item')


#Product id
product_id = [p.get('data-articlecode') for p in products_list]


#Product Category
product_category = [p.get('data-category') for p in products_list]

#Product name
products_list = products.find_all('a', class_= 'link')
product_name = [p.get_text() for p in products_list]

#Price
price_list = products.find_all('span', class_= 'price regular')
price = [ p.get_text() for p in price_list]
price

data = pd.DataFrame([product_id, product_category, price]).T
data.columns = ['product_id', 'product_category', 'Price']



#Datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d  %H:%M:%S' )
data.head(15)


#Pagination

total_item = soup.find_all('h2', class_= 'load-more-heading')[0].get('data-total')
total_item

#Page number
page_number = np.round(int(total_item) / 36)
page_number


#Concating page number with url

url02 = url + '?page-size' + str(int(page_number * 36))
url02





'https://www2.hm.com/en_us/men/products/jeans.html?page-size108'

In [2]:
type(product_id)

list

In [3]:
products_list

[<a class="link" href="/en_us/productpage.0690449022.html">Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0427159006.html">Trashed Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0636207006.html">Slim Jeans</a>,
 <a class="link" href="/en_us/productpage.0720504001.html">Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0636207010.html">Slim Jeans</a>,
 <a class="link" href="/en_us/productpage.0730863005.html">Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0720504008.html">Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0985159001.html">Skinny Jeans</a>,
 <a class="link" href="/en_us/productpage.0868115006.html">Skinny Cropped Jeans</a>,
 <a class="link" href="/en_us/productpage.0868115001.html">Skinny Cropped Jeans</a>,
 <a class="link" href="/en_us/productpage.0814631004.html">Freefit® Slim Jeans</a>,
 <a class="link" href="/en_us/productpage.0814631002.html">Freefit® Slim Jeans</a>,
 <a class="link" href="/en_us/productpage.

# Multiple Products

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# empty dataframe
df_details = pd.DataFrame()
# unique columns for all products
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )
for i in range( len( data ) ):
    
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id']+ '.html'
    page = requests.get( url, headers=headers )
    
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature' )
    color_name = [p.get( 'data-color' ) for p in product_list]
    
    
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    
    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    
    # ==================== composition =================================
    product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
    product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
    
    # reaname dataframe
    df_composition = pd.DataFrame( product_composition ).T
    df_composition.columns = df_composition.iloc[0]
    
    # delete first row
    df_composition = df_composition.iloc[1:].fillna( method='ffill' )
    
    # garantee the same number of columns
    df_composition = pd.concat( [df_pattern, df_composition], axis=0 )
    
    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
    df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )
    aux = aux + df_composition.columns.tolist()
    
    # merge data color + decomposition
    
    data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition', 'Size', 'Product safety']], how='left', on='style_id' )
    
    # all details products
    df_details = pd.concat( [df_details, data_sku], axis=0 )
    
    # Join Showroom data + details
    data['style_id'] = data['product_id'].apply( lambda x: x[:-3] )
    data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )
    data_raw = pd.merge( data, df_details[['style_id', 'color_name', 'Fit','Composition', 'Size', 'Product safety']], 
                        how='left', on='style_id' )

In [7]:
data_raw.head(50)

,product_id,product_category,Price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,Size,Product safety
0,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Light denim blue/trashed,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
1,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Light denim blue/trashed,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
2,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Denim blue,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
3,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Denim blue,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
4,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Black/washed,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
5,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Black/washed,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
6,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Light denim blue,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
7,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Light denim blue,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
8,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Black washed out,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
9,0690449022,men_jeans_skinny,$ 39.99,2021-06-12 14:35:38,0690449,022,Black washed out,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
